In [1]:
import boto3

# client = boto3.client("bedrock-runtime", region_name="us-west-2")
# model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"

session = boto3.Session(profile_name="bedrock-dev")
client = session.client("bedrock-runtime", region_name="us-west-2")


# Claude model ID — must be correct and available in the region
model_id = "us.anthropic.claude-3-sonnet-20240229-v1:0"

# Tool name for Claude 3.7
text_editor = "text_editor_20250124"  # this should be base on claude version.!!!

# Tool name for Claude 3.5
# text_editor = "text_editor_20241022"

In [2]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
    text_editor=None,
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools or text_editor:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    if text_editor:
        params["additionalModelRequestFields"] = {
            "tools": [
                {
                    "type": text_editor,
                    "name": "str_replace_editor",
                }
            ]
        }

    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [3]:
# Implementation of the TextEditorTool
import os
import shutil
from typing import Optional, List


class TextEditorTool:
    def __init__(self, base_dir: str = "", backup_dir: str = ""):
        self.base_dir = base_dir or os.getcwd()
        self.backup_dir = backup_dir or os.path.join(self.base_dir, ".backups")
        os.makedirs(self.backup_dir, exist_ok=True)

    def _validate_path(self, file_path: str) -> str:
        abs_path = os.path.normpath(os.path.join(self.base_dir, file_path))
        if not abs_path.startswith(self.base_dir):
            raise ValueError(
                f"Access denied: Path '{file_path}' is outside the allowed directory"
            )
        return abs_path

    def _backup_file(self, file_path: str) -> str:
        if not os.path.exists(file_path):
            return ""
        file_name = os.path.basename(file_path)
        backup_path = os.path.join(
            self.backup_dir, f"{file_name}.{os.path.getmtime(file_path):.0f}"
        )
        shutil.copy2(file_path, backup_path)
        return backup_path

    def _restore_backup(self, file_path: str) -> str:
        file_name = os.path.basename(file_path)
        backups = [
            f
            for f in os.listdir(self.backup_dir)
            if f.startswith(file_name + ".")
        ]
        if not backups:
            raise FileNotFoundError(f"No backups found for {file_path}")

        latest_backup = sorted(backups, reverse=True)[0]
        backup_path = os.path.join(self.backup_dir, latest_backup)

        shutil.copy2(backup_path, file_path)
        return f"Successfully restored {file_path} from backup"

    def _count_matches(self, content: str, old_str: str) -> int:
        return content.count(old_str)

    def view(
        self, file_path: str, view_range: Optional[List[int]] = None
    ) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if os.path.isdir(abs_path):
                try:
                    return "\n".join(os.listdir(abs_path))
                except PermissionError:
                    return "Error: Permission denied. Cannot list directory contents."

            if not os.path.exists(abs_path):
                return "Error: File not found"

            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            if view_range:
                start, end = view_range
                lines = content.split("\n")

                if end == -1:
                    end = len(lines)

                selected_lines = lines[start - 1 : end]

                result = []
                for i, line in enumerate(selected_lines, start):
                    result.append(f"{i}: {line}")

                return "\n".join(result)
            else:
                lines = content.split("\n")
                result = []
                for i, line in enumerate(lines, 1):
                    result.append(f"{i}: {line}")

                return "\n".join(result)

        except UnicodeDecodeError:
            return (
                "Error: File contains non-text content and cannot be displayed."
            )
        except ValueError as e:
            return f"Error: {str(e)}"
        except PermissionError:
            return "Error: Permission denied. Cannot access file."
        except Exception as e:
            return f"Error: {str(e)}"

    def str_replace(self, file_path: str, old_str: str, new_str: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                return "Error: File not found"

            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            match_count = self._count_matches(content, old_str)

            if match_count == 0:
                return "Error: No match found for replacement. Please check your text and try again."
            elif match_count > 1:
                return f"Error: Found {match_count} matches for replacement text. Please provide more context to make a unique match."

            # Create backup before modifying
            self._backup_file(abs_path)

            # Perform the replacement
            new_content = content.replace(old_str, new_str)

            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(new_content)

            return "Successfully replaced text at exactly one location."

        except ValueError as e:
            return f"Error: {str(e)}"
        except PermissionError:
            return "Error: Permission denied. Cannot modify file."
        except Exception as e:
            return f"Error: {str(e)}"

    def create(self, file_path: str, file_text: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            # Check if file already exists
            if os.path.exists(abs_path):
                return (
                    "Error: File already exists. Use str_replace to modify it."
                )

            # Create parent directories if they don't exist
            os.makedirs(os.path.dirname(abs_path), exist_ok=True)

            # Create the file
            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(file_text)

            return f"Successfully created {file_path}"

        except ValueError as e:
            return f"Error: {str(e)}"
        except PermissionError:
            return "Error: Permission denied. Cannot create file."
        except Exception as e:
            return f"Error: {str(e)}"

    def insert(self, file_path: str, insert_line: int, new_str: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                return "Error: File not found"

            # Create backup before modifying
            self._backup_file(abs_path)

            with open(abs_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            # Handle line endings
            if lines and not lines[-1].endswith("\n"):
                new_str = "\n" + new_str

            # Insert at the beginning if insert_line is 0
            if insert_line == 0:
                lines.insert(0, new_str + "\n")
            # Insert after the specified line
            elif insert_line > 0 and insert_line <= len(lines):
                lines.insert(insert_line, new_str + "\n")
            else:
                return f"Error: Line number {insert_line} is out of range. File has {len(lines)} lines."

            with open(abs_path, "w", encoding="utf-8") as f:
                f.writelines(lines)

            return f"Successfully inserted text after line {insert_line}"

        except ValueError as e:
            return f"Error: {str(e)}"
        except PermissionError:
            return "Error: Permission denied. Cannot modify file."
        except Exception as e:
            return f"Error: {str(e)}"

    def undo_edit(self, file_path: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                return "Error: File not found"

            return self._restore_backup(abs_path)

        except ValueError as e:
            return f"Error: {str(e)}"
        except FileNotFoundError:
            return "Error: No previous edits to undo"
        except PermissionError:
            return "Error: Permission denied. Cannot restore file."
        except Exception as e:
            return f"Error: {str(e)}"


In [4]:
# Process Tool Call Requests
import json

text_editor_tool = TextEditorTool()


def run_tool(tool_name, tool_input):
    if tool_name == "str_replace_editor":
        command = tool_input.get("command", "")
        if command == "view":
            path = tool_input.get("path", "")
            view_range = tool_input.get("view_range", None)
            return text_editor_tool.view(path, view_range)
        elif command == "str_replace":
            path = tool_input.get("path", "")
            old_str = tool_input.get("old_str", "")
            new_str = tool_input.get("new_str", "")
            return text_editor_tool.str_replace(path, old_str, new_str)
        elif command == "create":
            path = tool_input.get("path", "")
            file_text = tool_input.get("file_text", "")
            return text_editor_tool.create(path, file_text)
        elif command == "insert":
            path = tool_input.get("path", "")
            insert_line = tool_input.get("insert_line", 0)
            new_str = tool_input.get("new_str", "")
            return text_editor_tool.insert(path, insert_line, new_str)
        elif command == "undo_edit":
            path = tool_input.get("path", "")
            return text_editor_tool.undo_edit(path)
        else:
            raise Exception(f"Unknown text editor command: {command}")
    else:
        raise Exception(f"Unknown tool name: {tool_name}")


def run_tools(parts):
    tool_requests = [part for part in parts if "toolUse" in part]
    tool_result_parts = []

    for tool_request in tool_requests:
        tool_use_id = tool_request["toolUse"]["toolUseId"]
        tool_name = tool_request["toolUse"]["name"]
        tool_input = tool_request["toolUse"]["input"]

        try:
            tool_output = run_tool(tool_name, tool_input)
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [{"text": json.dumps(tool_output)}],
                    "status": "success",
                }
            }
        except Exception as e:
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [{"text": f"Error: {e}"}],
                    "status": "error",
                }
            }

        tool_result_parts.append(tool_result_part)

    return tool_result_parts


In [5]:
# Workaround for AWS. To include ToolUse message parts (which are required for the TextEditor tool),
# you must include at least one tool schema.

forbidden_tool_schema = {
    "toolSpec": {
        "name": "forbidden_tool",
        "description": "This tool is deprecated and should not be called under any circumstances. It has been replaced by newer alternatives and may cause system instability or data corruption if used. This schema exists only for documentation and backward compatibility purposes.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "parameter": {
                        "type": "string",
                        "description": "This parameter is no longer supported. Do not attempt to use this tool.",
                    }
                },
            }
        },
    }
}

In [6]:
# Run the conversation in a loop until the model doesn't ask for a tool use
def run_conversation(messages):
    while True:
        result = chat(
            messages,
            text_editor=text_editor,
            tools=[forbidden_tool_schema],
        )

        add_assistant_message(messages, result["parts"])
        print(result["text"])

        if result["stop_reason"] != "tool_use":
            break

        tool_result_parts = run_tools(result["parts"])
        add_user_message(messages, tool_result_parts)

    return messages

In [ ]:
messages = []

add_user_message(
    messages,
    """
    write a one sentence description of the code in the ./main.py file
    """,
)

#    """
#    In the ./main.py write out a function to calculate pi to the 5th digit
#     then make a ./test.py file to test out that function
#    """,

run_conversation(messages)

ValidationException: An error occurred (ValidationException) when calling the Converse operation: The additional field tools conflicts with an existing field. Remove tools and try again.